In [1]:
import pandas as pd
import numpy as np

# 데이터 만들기

In [37]:
convini_A = pd.Series([1,4,3,3,3,3,3])
convini_B = pd.Series([4,4,3,4,4,5,4,4])
convini_C = pd.Series([4,3,4,3,4,4,3,3,3])

In [44]:
# 열 기준으로 붙이면 생기는 nan때문에 계산이 어려울 때가 있다.

convini_total = pd.concat([convini_A, convini_B, convini_C], axis = 0)


# 통계량 계산하기

In [69]:
# SST - 총제곱합

mean_total = convini_total.mean()
SST = np.sum((convini_total - mean_total)**2)

print(mean_total, SST)

3.4583333333333335 13.958333333333334


In [52]:
# SSB - 집단 간 편차

mean_A = convini_A.mean()
mean_B = convini_B.mean()
mean_C = convini_C.mean()

mean_lst = [mean_A, mean_B, mean_C]
convi_len = [len(convini_A), len(convini_B), len(convini_C)]

SSB = sum(convi_len * (mean_lst - mean_total)**2)

print(mean_A, mean_B, mean_C, SSB)

2.857142857142857 4.0 3.4444444444444446 4.878968253968253


In [54]:
# SSW - 집단 내 편차

samples_within = [
    sum((convini_A - mean_A) ** 2),
    sum((convini_B - mean_B) ** 2),
    sum((convini_C - mean_C) ** 2)  
]

SSW = sum(samples_within)

print(SSW)

9.079365079365079


In [55]:
# SST 검증

print(SST, SSB + SSW)

13.958333333333334 13.958333333333332


# 일원분산분석의 결과 해석

In [58]:
# 집단의 갯수는 3개, 따라서 자유도는 2, 21

MSB = SSB / (3 - 1)
MSW = SSW / (len(convini_total) - 3)
F_value = MSB / MSW

print(MSB, MSW, F_value)

2.4394841269841265 0.43235071806500375 5.642373251748251


# 통계 검증

In [68]:
from scipy import stats

# F-value와 자유도 설정
df_between = 3 - 1  # 집단 간 자유도 (k-1, k는 집단의 수)
df_within = len(convini_total) - 3  # 집단 내 자유도 (N-k, N은 총 샘플 수)

print(round(F_value, 4), df_between, df_within)

# p-값 계산
p_value = stats.f.sf(F_value, df_between, df_within)

if p_value < 0.05:
    print(f'P값: {p_value}, 귀무가설을 기각할 충분한 통계적 근거가 있다.')
else:
    print(f'P값: {p_value}, 귀무가설을 기각할 충분한 통계적 근거가 부족하다.')

5.6424 2 21
P값: 0.010935284769920764, 귀무가설을 기각할 충분한 통계적 근거가 있다.


# 이원분산분석

In [72]:
convini_data = pd.DataFrame({
    'region': ('A', 'A','A', 'B', 'B', 'B', 'C', 'C', 'C'),
    'convini_A': (1,4,1,2,2,3,2,3,2),
    'convini_B': (4,4,3,3,2,3,4,2,3),
    'convini_C': (4,3,4,4,3,3,2,4,4)
})

convini_data

,region,convini_A,convini_B,convini_C
0,A,1,4,4
1,A,4,4,3
2,A,1,3,4
3,B,2,3,4
4,B,2,2,3
5,B,3,3,3
6,C,2,4,2
7,C,3,2,4
8,C,2,3,4


In [116]:
# SST 구하기

total_mean = convini_data[['convini_A', 'convini_B', 'convini_C']].mean().mean()
SST = sum((convini_data[['convini_A', 'convini_B', 'convini_C']].values.flatten() - total_mean)**2)

print(total_mean, SST)

2.925925925925926 23.851851851851862


In [117]:
# SSA - 편의점별 제곱합 구하기

len_convi = [len(convini_data['convini_A']), len(convini_data['convini_B']), len(convini_data['convini_C'])]
mean_convi = [convini_data['convini_A'].mean(), convini_data['convini_B'].mean(), convini_data['convini_C'].mean()]

SSA = sum(len_convi * (mean_convi - total_mean)**2)

print(len_convi)
print(mean_convi)
print(SSA)


[9, 9, 9]
[np.float64(2.2222222222222223), np.float64(3.111111111111111), np.float64(3.4444444444444446)]
7.185185185185185


In [118]:
# SSB - 지역별 제곱합 구하기

len_region = convini_data.groupby('region').size()*3
mean_region = convini_data.groupby('region').mean().mean(axis=1)

SSB = sum(len_region * (mean_region - total_mean)**2)

print(len_region)
print(mean_region)
print(SSB)

region
A    9
B    9
C    9
dtype: int64
region
A    3.111111
B    2.777778
C    2.888889
dtype: float64
0.518518518518516


In [151]:
#SSAB - 상호작용의 잔차제곱합

mean_1 = [2,3.667,3.667]
mean_2 = [2.333, 2.667, 3.333]
mean_3 = [2.333, 3, 3.333]

# SSAB - 상호작용의 잔차제곱합 계산 수정
# 각 위치의 값들끼리 계산
ssab_1 = sum(3 * ((mean_1[i] - mean_convi[i] - mean_region[0] + total_mean) ** 2) for i in range(len(mean_1)))
ssab_2 = sum(3 * ((mean_2[i] - mean_convi[i] - mean_region[1] + total_mean) ** 2) for i in range(len(mean_2)))
ssab_3 = sum(3 * ((mean_3[i] - mean_convi[i] - mean_region[2] + total_mean) ** 2) for i in range(len(mean_3))) 

SSAB = sum([ssab_1, ssab_2, ssab_3])
print(SSAB)

1.4809652962962967


C:\Users\campus3S031\AppData\Local\Temp\ipykernel_67572\1298032322.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ssab_1 = sum(3 * ((mean_1[i] - mean_convi[i] - mean_region[0] + total_mean) ** 2) for i in range(len(mean_1)))
C:\Users\campus3S031\AppData\Local\Temp\ipykernel_67572\1298032322.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ssab_2 = sum(3 * ((mean_2[i] - mean_convi[i] - mean_region[1] + total_mean) ** 2) for i in range(len(mean_2)))
C:\Users\campus3S031\AppData\Local\Temp\ipykernel_67572\1298032322.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ve

In [152]:
# SSW

SSW = SST - SSA - SSB - SSAB
print(SSW)

14.667182851851866


# 이원분산분석의 결과 해석

In [155]:
# 평균잔차제곱합들 구하기

MSA = SSA / (3 - 1)
MSB = SSB / (3 - 1)
MSAB = SSAB / (2 * 2)
MSW = SSW / (3*3*(3-1))

print('MSA:', MSA)
print('MSB:', MSB)
print('MSAB:', MSAB)
print('MSW:', MSW)

MSA: 3.5925925925925926
MSB: 0.259259259259258
MSAB: 0.3702413240740742
MSW: 0.8148434917695481


In [164]:
from scipy import stats

F_region = MSB / MSW  # 지역 F-통계량
F_store = MSA / MSW  # 편의점 F-통계량
F_interaction = MSAB / MSW  # 상호작용 F-통계량

print('------각각 지역, 편의점, 상호작용에 대한 F통계량------', '\n')
print(F_region, F_store, F_interaction, '\n')

p_value_region = 1 - stats.f.cdf(F_region, 2, 26)  # 지역에 대한 p값
p_value_store = 1 - stats.f.cdf(F_store, 2, 26)  # 편의점에 대한 p값
p_value_interaction = 1 - stats.f.cdf(F_interaction, 4, 26)  # 상호작용에 대한 p값

print('------각각 지역, 편의점, 상호작용에 대한 검증------', '\n')
for p_value in [p_value_region, p_value_store, p_value_interaction]:
    if p_value < 0.05:
        print(f'P값: {p_value}, 귀무가설을 기각할 충분한 통계적 근거가 있다.')
    else:
        print(f'P값: {p_value}, 귀무가설을 기각할 충분한 통계적 근거가 부족하다.')

------각각 지역, 편의점, 상호작용에 대한 F통계량------ 

0.31817062034359483 4.408935739046978 0.45437108820743316 

------각각 지역, 편의점, 상호작용에 대한 검증------ 

P값: 0.7302710982994091, 귀무가설을 기각할 충분한 통계적 근거가 부족하다.
P값: 0.022450431442032626, 귀무가설을 기각할 충분한 통계적 근거가 있다.
P값: 0.7683249065985003, 귀무가설을 기각할 충분한 통계적 근거가 부족하다.
